# Classificador de Respostas de Atendimento
Notebook completo para treinar um modelo BERTimbau que determina se a **resposta do analista** é *completa* ou *incompleta* levando em conta o texto da **manifestação do cliente**.

**Pré‑requisitos**  
- Python 3.8+  
- GPU (opcional, mas recomendado)  
- Pacotes: `transformers`, `datasets`, `scikit-learn`, `pandas`, `torch`

Ajuste os caminhos dos arquivos conforme necessário.

In [ ]:
# Se necessário, descomente para instalar dependências
# !pip install transformers datasets scikit-learn pandas torch --upgrade


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch, joblib, os, json

## Carregar a base de dados

In [ ]:
# Substitua pelo caminho do seu CSV
DATA_PATH = 'base_respostas.csv'  # ex.: '/content/drive/MyDrive/base_respostas.csv'

df = pd.read_csv(DATA_PATH)

# Espera colunas: 'manifestacao', 'resposta', 'classificacao'
df.head()

## Pré‑processamento

In [ ]:
# Converter NaNs para string vazia e criar coluna pareada
df['manifestacao'] = df['manifestacao'].fillna('')
df['resposta']     = df['resposta'].fillna('')
df['pair'] = df['manifestacao'] + ' [SEP] ' + df['resposta']

# Converter target para numérico: 'resposta completa' -> 1, outras -> 0
df['label'] = (df['classificacao'].str.lower().str.contains('completa')).astype(int)

df[['manifestacao','resposta','label']].head()

## Separar treino e teste

In [ ]:
train_df, test_df = train_test_split(df[['pair','label']],
                               test_size=0.2,
                               stratify=df['label'],
                               random_state=42)

len(train_df), len(test_df)

## Tokenizar com BERTimbau

In [ ]:
MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(batch['pair'],
                     truncation=True,
                     padding='max_length',
                     max_length=256)

train_ds = Dataset.from_pandas(train_df).map(tokenize, batched=True)
test_ds  = Dataset.from_pandas(test_df).map(tokenize, batched=True)

# Remover colunas redundantes para economizar memória
cols = ['pair']
train_ds = train_ds.remove_columns(cols)
test_ds  = test_ds.remove_columns(cols)

train_ds, test_ds

## Treinar o modelo

In [ ]:
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

training_args = TrainingArguments(
    output_dir='bert_respostas',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

trainer.train()

## Avaliação

In [ ]:
preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = preds.predictions.argmax(axis=-1)

print(classification_report(y_true, y_pred, target_names=['incompleta','completa']))
print('Matriz de confusão:\n', confusion_matrix(y_true, y_pred))

## Salvar modelo e tokenizer

In [ ]:
SAVE_DIR = 'modelo_resposta_completa'
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

# Também salvar como joblib para integração mais leve
joblib.dump({'model_path': SAVE_DIR}, 'config_modelo.joblib')
print(f'Modelo salvo em {SAVE_DIR}')

## Função de inferência

In [ ]:
def classificar_resposta(manifestacao: str, resposta: str, model_dir: str = SAVE_DIR):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = BertForSequenceClassification.from_pretrained(model_dir).to(device)
    tokenizer = BertTokenizerFast.from_pretrained(model_dir)
    texto = manifestacao + ' [SEP] ' + resposta
    inputs = tokenizer(texto, return_tensors='pt', truncation=True, padding='max_length', max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probas = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    classe = 'resposta completa' if probas[1] > probas[0] else 'resposta incompleta'
    return classe, probas

# Exemplo rápido
ex_manifestacao = """O boleto está vencido e não consigo gerar outro."""
ex_resposta = """Entramos em contato por e‑mail e enviamos a 2ª via do boleto, com novo vencimento em 10/07/2025. Caso precise de outra data, estamos à disposição."""
classe, probas = classificar_resposta(ex_manifestacao, ex_resposta)
print(classe, probas)